In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import numpy as np
import json


In [ ]:
driver = webdriver.Chrome()  
driver.get('https://hoopr.ai/search?q=&t=music')
time.sleep(30)
last_height = driver.execute_script("return document.body.scrollHeight")
max_iterations = 100  
current_iteration = 0


while current_iteration < max_iterations:
    print(f"Scrolling iteration: {current_iteration + 1}")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)  
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        print("No more new content to load.")
        break
    last_height = new_height
    current_iteration += 1
driver.execute_script("window.scrollTo(0, 0);")
driver.execute_script(f"window.scrollBy(0, {200});")


In [ ]:
def process_track(item,k):
    """Process a single track entry and extract details."""
    try:
        data = item.text.strip()
        print(f"Processing: {data}")

        
        track_info = data.split(" ×Share Track")[0].strip()

        
        parts = track_info.split()

        
        if len(parts) < 3:
            print(f"Skipping entry due to insufficient data: {track_info}")
            return None

        
        song_name = " ".join(parts[:2])

        
        artist_index = 2
        while artist_index < len(parts) and not parts[artist_index].replace(":", "").isdigit():
            artist_index += 1

        
        if artist_index >= len(parts):
            print(f"Skipping entry due to missing duration: {track_info}")
            return None

        artist = " ".join(parts[2:artist_index])

        
        duration = parts[artist_index]

        
        genres = " ".join(parts[artist_index + 1:]).split("  ")
        main_genre = genres[0].strip() if genres else "Unknown"

        
        return {
            "Song Name": song_name,
            "Artist": artist,
            "Duration": duration,
            "Main Genre": main_genre,
            "audio-link":str(k)
        }
    except Exception as e:
        print(f"Error processing entry: {item.text}\n{e}")
        return None

In [ ]:
track_data=[]
soup = BeautifulSoup(driver.page_source, 'html.parser')


print("Finding and clicking buttons within containers...")
containers = driver.find_elements(By.CLASS_NAME, 'ng-star-inserted')  

for index in range(0, len(containers), 2):  
    container = containers[index]
    try:
        container_html = container.get_attribute('outerHTML')
         
    
        soup = BeautifulSoup(container_html, 'html.parser')
        
        button = container.find_element(By.CLASS_NAME, 'play-pause-icon')
        
        
        button.click()
        time.sleep(1)  
        
        
        updated_soup = BeautifulSoup(driver.page_source, 'html.parser')
        updated_global_tags = updated_soup.find('audio', {'preload': 'auto'})  
        print("Updated global tags:")
        print(updated_global_tags)
        
        track = process_track(soup,updated_global_tags['src'])
        if track:
          track_data.append(track)
        driver.execute_script(f"window.scrollBy(0, {90});")
        
         
    except Exception as e:
        print(f"Error while clicking or extracting: {e}")
        
driver.quit()

In [ ]:
output_file = "track_data.json"
with open(output_file, "w", encoding="utf-8") as json_file:
    json.dump(track_data, json_file, indent=4, ensure_ascii=False)

print(f"Data saved to '{output_file}'.")

In [ ]:
print(len(track_data))

In [2]:
import json
import pandas as pd

# Read data from track_data.json
input_file = "track_data.json"
with open(input_file, "r", encoding="utf-8") as json_file:
    track_data = json.load(json_file)

# Process data for Excel
processed_data = []
for track in track_data:
    genres = track["Main Genre"].split(", ")
    processed_data.append({
        "Song Name": track["Song Name"],
        "Artist": track["Artist"],
        "Duration": track["Duration"],
        "audio-link": track["audio-link"],
        "Genre": genres[0] if len(genres) > 0 else "",
        "Mood1": genres[1] if len(genres) > 1 else "",
        "Mood2": genres[2] if len(genres) > 2 else ""
    })

# Create a DataFrame
df = pd.DataFrame(processed_data)

# Save to Excel
output_file = "track_data.xlsx"
df.to_excel(output_file, index=False)

print(f"Data saved to '{output_file}'.")


Data saved to 'track_data.xlsx'.
